In [1]:
""""
import torch
from torch import nn
#import torch.nn.functional as F
#import torch.optim as optim
#from torch.utils.data import Dataset, DataLoader
#import numpy as np
#from tqdm import tqdm, tqdm_notebook
"""""
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [2]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 2070'

In [3]:
# from kobert.utils import get_tokenizer
# from kobert.pytorch_kobert import get_pytorch_kobert_model

In [4]:
# bertmodel, vocab = get_pytorch_kobert_model()

In [5]:
# data_train = BERTDataset(dataset_train[:100], 0, 1, tok, max_len, True, False)

In [6]:
# data_train

In [7]:
# tokenizer = get_tokenizer()
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [3]:
##GPU 사용 시
device = torch.device("cuda:0")

In [4]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
from transformers import DistilBertModel

In [6]:
distilbert_model = DistilBertModel.from_pretrained('monologg/distilkobert')

In [7]:
distilbert_model

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(in

In [8]:
from tokenization_kobert import KoBertTokenizer

In [9]:
tokenizer = KoBertTokenizer.from_pretrained('monologg/distilkobert')

In [10]:
tokenizer

PreTrainedTokenizer(name_or_path='monologg/distilkobert', vocab_size=8002, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [11]:
import pandas as pd

In [12]:
df_data = pd.read_csv('../labeling/data/data3.csv')

In [13]:
df_data

,text_headline,label
0,사설 北 핵·ICBM 시험 재개 시사… ‘잘못된 길’ 선택해선 안 된다,1
1,사설 수출 역성장 멈추려면 기업 활력부터 되살려내야,2
2,사설 여권 일방통행식 국회 운영 정국 정상화 가능하겠나,2
3,설왕설래 인구절벽,1
4,세계포럼 친중 정책 레드라인 넘지 말아야,2
...,...,...
6091103,고무줄 면역효과 불구 영국은 왜 아스트라제네카를 승인했나,2
6091104,박원순의 마지막 이틀 부인→고백→대응→포기 흔들렸다,2
6091105,경기도 공공배달앱 ‘배달특급’ 20여일만에 거래액 20억 돌파,2
6091106,알림뉴시스 콘텐츠 저작권 고지,0


In [14]:
import re

In [16]:
data_list = []
for q, label in tqdm(zip(df_data['text_headline'], df_data['label'])):
    data = []
    clean_title = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…\"\“》]', ' ', str(q)) 
    data.append(clean_title)
    data.append(str(label))

    data_list.append(data)

6091108it [00:15, 389338.61it/s]


In [17]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[19999])
print(data_list[-1])

['사설 北 핵·ICBM 시험 재개 시사   잘못된 길’ 선택해선 안 된다', '1']
['돋을새김 전광훈과 한국당', '1']
['이총리 총선 탄핵 전후 분출된 국민요구 해결·지체의 분수령', '1']
['2030 리스펙트 미디어 정치와 정당의 위기  허승규', '1']
['새해 첫날 전국 가끔 구름 많고 일부 중부지방에 눈발', '1']
['녹유 오늘의 운세51년생 침이 고여지는 대접을 받아요', '0']


In [18]:
#데이터 셋 test와 train으로 4:1분리
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [19]:
import time

In [24]:
# class BERTDataset(Dataset):
#     def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
#                  pad, pair):
#         start = time.time()
#         transform = nlp.data.BERTSentenceTransform(
#             bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

#         self.sentences = [transform([i[sent_idx]]) for i in dataset]
#         self.labels = [np.int32(i[label_idx]) for i in dataset]
#         end = time.time()
#         print("Using time : ", round(end-start,2), end='\n')
#     def __getitem__(self, i):
#         return (self.sentences[i] + (self.labels[i], ))

#     def __len__(self):
#         return (len(self.labels))

In [20]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        tokens, masks, segments, targets = [], [], [], []
        
        start = time.time()
        # 문장을 토큰화
    
        # tokens = [bert_tokenizer.encode(d[sent_idx], max_length = max_len, pad_to_max_length=pad) for d in dataset]
        sentenses = [sentense[0] for sentense in dataset]

        tokens = bert_tokenizer.batch_encode_plus(sentenses, max_length = max_len, padding = pad)
       
        # 마스크 토큰화 한 부분 - 1 아닌 부분 0
        # num_zeros = [t.count(1) for t in tokens]
        

        # masks = [[1] * (max_len-num_zero) + [0] * num_zero for num_zero in num_zeros]

            # 문장의 전후 관계를 구분해주는 세그먼트는 문장이 1개 밖에 없으므로 모두 0

        # segments = [np.int32(max_len) for i in range(len(tokens))]

        targets = [np.int32(d[label_idx]) for d in dataset] 
        
        masks = tokens['attention_mask']
        
        valid_length = [mask.count(1) for mask in masks]
        
        # segments = tokens['token_type_ids']
        tokens = tokens['input_ids']
         
        
        self.valid_length = np.array(valid_length)
        self.tokens = np.array(tokens)
        self.masks = np.array(masks)
        # self.segments = np.array(segments)
        self.targets = np.array(targets)
        
        end = time.time()
        print("Using time : ", round(end-start,2), end ='\n')
        
    def __getitem__(self, i):
        return (self.tokens[i], self.valid_length[i], self.masks[i], self.targets[i])

    def __len__(self):
        return (len(self.targets))

In [21]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 1000
learning_rate =  5e-5

In [22]:
data_train = BERTDataset(dataset_train[:100], 0, 1, tokenizer, max_len, True, False)

Using time :  0.01


In [23]:
data_train[1]

(array([   2, 3332, 7234, 5495, 4446, 5330, 5760, 4487,  517, 7169, 7823,
        3517,    3,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1]),
 13,
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]),
 2)

In [24]:
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, max_len, True, False)


Using time :  489.48
Using time :  157.47


In [25]:
data_train[1]

(array([   2, 3332, 7234, 5495, 4446, 5330, 5760, 4487,  517, 7169, 7823,
        3517,    3,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1]),
 13,
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
     

In [26]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=0)

In [27]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        # _, pooler = self.bert(input_ids = token_ids, attention_mask = attention_mask.float().to(token_ids.device))
        output = self.bert(input_ids = token_ids, attention_mask = attention_mask.float().to(token_ids.device))
        # print(output[0][:,0])
        # print(output)
        if self.dr_rate:
            out = self.dropout(output[0][:,0])
        return self.classifier(out)

In [28]:
model = BERTClassifier(distilbert_model,  dr_rate=0.5).to(device)

In [29]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [30]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [31]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [32]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [33]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    # print(max_vals, max_indices,Y)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    predict_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        # segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        # print(token_ids, valid_length)
        out = model(token_ids, valid_length)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        # segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    torch.save(model, '../model/distilmodel_1_{}.pt'.format(e+2))
    

C:\Users\jeon\anaconda3\envs\stockpredict\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/71381 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.1475244015455246 train acc 0.953125
epoch 1 batch id 1001 loss 0.1340157836675644 train acc 0.9549825174825175
epoch 1 batch id 2001 loss 0.011229909025132656 train acc 0.9552411294352824
epoch 1 batch id 3001 loss 0.04438058286905289 train acc 0.9555252415861379
epoch 1 batch id 4001 loss 0.09414537250995636 train acc 0.9555657960509872
epoch 1 batch id 5001 loss 0.07994286715984344 train acc 0.9559775544891022
epoch 1 batch id 6001 loss 0.11352035403251648 train acc 0.9561375187468755
epoch 1 batch id 7001 loss 0.054013825953006744 train acc 0.9560955577774604
epoch 1 batch id 8001 loss 0.19321367144584656 train acc 0.9562359392575928


In [35]:
torch.save(model, '../model/distilmodel_1_{}.pt'.format(e+1))